In [38]:
!pip install geopandas

import pandas
import numpy
import geopandas as gpd

import bs4
import re

import matplotlib.pyplot as plt
%matplotlib inline

from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

from functools import reduce

**Importation des bases de données contenant les indicateurs suivants :**
- Le niveau de diplôme et de formation par commune (data_diplome);
- La catégorie socio-professionnelle par commune (data_csp);
-	Le taux d’activité par tranche d’âge par commune (data_activite); (**OK**)
- La médiane du niveau de vie par département (data_mediane); (**OK**)
-	Le salaire net horaire moyen par département (data_salhor). (**OK**)





La fonction suivante (**csv_zip_to_df**) permet de convertir des un **fichier CSV en format ZIP** grâce à un URL et au nom du fichier que celui-ci contient en un **dataframe**.

# **Importation des données et constitution de la DataFrame**


In [ ]:
def csv_zip_to_df (url_file,file_name):
  url = urlopen(url_file)
  zipfile = ZipFile(BytesIO(url.read()))
  dataframe = pd.read_csv(zipfile.open(file_name), header = 0,sep=";")
  return dataframe

## **Base comparateur des territoires en 2017 - INSEE**
Cette base, issue de l'INSEE (https://www.insee.fr/fr/statistiques/2521169) renvoie des **indicateurs économiques** à l'échelle communale. Pour notre étude, nous n'avons pas besoin de l'intégralité des variables présentes sur cette base, nous allons en sélectionner quelques unes : 

- Population en 2017 (**P17_POP**);
- Médiane du niveau vie en 2017 (**MED17**);
- Emplois au LT en 2017 (**P17_EMPLT**) ; 

Et en créer certaines :
- Densité en 2017 (**DEN17**);
- Taux d'activité en 2017 (**T17_ACT1564**);
- Taux de chômage en 2017 (**T17_CHOM1564**).

In [109]:
bdd_ind_eco = csv_zip_to_df("https://www.insee.fr/fr/statistiques/fichier/2521169/base-comparateur-2017_CSV.zip","base_cc_comparateur.CSV")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [110]:
#Création du taux de chômage et d'activité, et de la densité
bdd_ind_eco['T17_ACT1564'] = bdd_ind_eco['P17_ACT1564'] / bdd_ind_eco['P17_POP1564']
bdd_ind_eco['T17_CHOM1564'] = bdd_ind_eco['P17_CHOM1564'] / bdd_ind_eco['P17_POP1564']
bdd_ind_eco['DEN17'] = bdd_ind_eco ['P17_POP'] / bdd_ind_eco['SUPERF']

#Sélection de variables et arrondis
bdd_ind_eco = bdd_ind_eco[['CODGEO','P17_POP','DEN17','MED17','P17_EMPLT','T17_ACT1564','T17_CHOM1564']]
bdd_ind_eco[['DEN17','P17_EMPLT']]=bdd_ind_eco[['DEN17','P17_EMPLT']].round(decimals=0)

bdd_ind_eco

,CODGEO,P17_POP,DEN17,MED17,P17_EMPLT,T17_ACT1564,T17_CHOM1564
0,01001,776.0,49.0,23310.0,99.0,0.812070,0.071370
1,01002,248.0,27.0,24290.0,16.0,0.854167,0.069444
2,01004,14035.0,571.0,19860.0,7686.0,0.762468,0.113257
3,01005,1689.0,106.0,23370.0,294.0,0.807161,0.063349
4,01006,111.0,19.0,23970.0,6.0,0.805556,0.111111
...,...,...,...,...,...,...,...
34994,75116,166361.0,10163.0,38900.0,118109.0,0.732103,0.073431
34995,75117,167288.0,29504.0,30800.0,106754.0,0.798800,0.090348
34996,75118,195233.0,32485.0,22330.0,88117.0,0.801058,0.101803
34997,75119,187015.0,27543.0,20320.0,89181.0,0.766736,0.126026


## **Base salaire net horaire moyen en 2017 - INSEE**
Cette base, issue de l'INSEE (https://www.insee.fr/fr/statistiques/2021266) renvoie des indicateurs sur le **salaire horaire net moyen par tranches d'âge, par sexes et par catégories socio-professionnelles**, à l'échelle communale. Nous allons garder les variables suivantes : 

- Salaire net horaire moyen en 2017 (€) (**SNHM17**);
- Salaire net horaire moyen F en 2017 (€) (**SNHMF17**);
- Salaire net horaire moyen H en 2017 (€) (**SNHMH17**);
- Salaire net hor. Moy. Cadres sup. en 2017 (€) (**SNHMC17**);
- Salaire net hor. moy. prof inter. en 2017 (€) (**SNHMP17**);
- Salaire net hor. moy. employés en 2017 (€) (**SNHME17**);
- Salaire net hor. moy. ouvriers en 2017 (€) (**SNHMO17**);
- Salaire net horaire moyen 18 à 25 ans en 2017 (€) (**SNHM1817**);
- Salaire net horaire moyen 26 à 50 ans en 2017 (€) (**SNHM2617**);
- Salaire net horaire moyen plus de 50 ans en 2017 (€) (**SNHM5017**).







In [107]:
bdd_sal = csv_zip_to_df ("https://www.insee.fr/fr/statistiques/fichier/2021266/base-cc-dads-2017_csv.zip","cc_dads_2017_COM.csv")

In [108]:
#Sélection de variables et arrondis
bdd_sal = bdd_sal [['CODGEO','SNHM17','SNHMF17','SNHMH17','SNHMC17','SNHMP17','SNHME17','SNHMO17','SNHM1817','SNHM2617','SNHM5017']].round(decimals=2)

bdd_sal

,CODGEO,SNHM17,SNHMF17,SNHMH17,SNHMC17,SNHMP17,SNHME17,SNHMO17,SNHM1817,SNHM2617,SNHM5017
0,01004,14.18,12.12,15.40,24.74,15.97,10.46,11.10,10.17,14.25,16.61
1,01007,13.67,12.53,14.59,21.95,14.58,10.72,11.16,9.90,13.84,14.65
2,01014,15.30,11.84,17.62,39.89,15.71,11.15,10.94,10.39,15.74,15.99
3,01024,13.15,11.71,14.16,22.10,14.77,10.92,11.12,9.73,13.34,14.18
4,01025,13.37,12.25,14.09,23.12,14.48,10.80,11.27,9.45,13.27,15.17
...,...,...,...,...,...,...,...,...,...,...,...
5358,75116,38.32,26.07,49.34,51.93,32.35,12.60,11.57,20.32,34.10,51.91
5359,75117,26.85,22.67,30.76,36.37,17.54,12.10,11.21,12.03,26.14,34.93
5360,75118,18.60,17.92,19.15,26.50,15.67,11.34,10.62,11.06,18.88,20.86
5361,75119,16.44,15.89,16.86,24.78,15.25,11.08,10.46,10.40,16.70,17.95


## **Base scolarisation en 2017 - INSEE**
Cette base, issue de l'INSEE (https://www.insee.fr/fr/statistiques/4516086?sommaire=4516089) renvoie des indicateurs sur la **scolarisation par catégories d'âge**, à l'échelle communale. Nous allons sélectionner certaines variables afin de créer les suivantes :
- Taux de scolarisation entre 2 et 5 ans (**T17_0205**);
- Taux de scolarisation entre 6 et 10 ans (**T17_0610**);
- Taux de scolarisation entre 11 et 14 ans (**T17_1114**);
- Taux de scolarisation entre 15 et 17 ans (**T17_1517**);
- Taux de scolarisation entre 18 et 24 ans (**T17_1824**);
- Taux de scolarisation entre 25 et 29 ans (**T17_2529**).

In [112]:
bdd_scolar = csv_zip_to_df("https://www.insee.fr/fr/statistiques/fichier/4516086/base-ccc-diplomes-formation-2017.zip","base-cc-diplomes-formation-2017.CSV")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [114]:
#Création des taux de scolarisation
def taux(new_var,nom_var,denom_var,bdd):
  bdd[new_var] = bdd[nom_var] / bdd[denom_var]
  return bdd[new_var]

taux('T17_0205','P17_SCOL0205','P17_POP0205',bdd_scolar)
taux('T17_0610','P17_SCOL0610','P17_POP0610',bdd_scolar)
taux('T17_1114','P17_SCOL1114','P17_POP1114',bdd_scolar)
taux('T17_1517','P17_SCOL1517','P17_POP1517',bdd_scolar)
taux('T17_1824','P17_SCOL1824','P17_POP1824',bdd_scolar)
taux('T17_2529','P17_SCOL2529','P17_POP2529',bdd_scolar)

#Sélection de variables
bdd_scolar = bdd_scolar [['CODGEO','T17_0205','T17_0610','T17_1114','T17_1517','T17_1824','T17_2529']]

bdd_scolar

,CODGEO,T17_0205,T17_0610,T17_1114,T17_1517,T17_1824,T17_2529
0,1001,0.678734,0.964721,1.000000,0.969704,0.554161,0.060618
1,1002,0.687500,1.000000,1.000000,1.000000,0.375000,0.076923
2,1004,0.718624,0.989293,0.988981,0.953482,0.386842,0.025314
3,1005,0.723092,0.974497,0.976849,0.948244,0.367987,0.000000
4,1006,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
34991,97420,0.764037,0.991133,0.989295,0.943619,0.365607,0.025515
34992,97421,0.720430,0.962775,0.972103,0.897361,0.224739,0.019704
34993,97422,0.730053,0.974402,0.984340,0.935708,0.401891,0.037657
34994,97423,0.804610,0.987735,0.987336,0.930096,0.274024,0.028133


## **Consolidation des DataFrames**

In [131]:
#Création d'une liste contenant les dataframes à consolider
liste_df = [bdd_ind_eco, bdd_sal, bdd_scolar]

df_final = reduce(lambda left,right: pd.merge(left, right, on='CODGEO',left_index=True, right_index=True, how='outer'), liste_df)

df_final

,CODGEO,P17_POP,DEN17,MED17,P17_EMPLT,T17_ACT1564,T17_CHOM1564,SNHM17,SNHMF17,SNHMH17,SNHMC17,SNHMP17,SNHME17,SNHMO17,SNHM1817,SNHM2617,SNHM5017,T17_0205,T17_0610,T17_1114,T17_1517,T17_1824,T17_2529
0,01001,776.0,49.0,23310.0,99.0,0.812070,0.071370,14.18,12.12,15.40,24.74,15.97,10.46,11.10,10.17,14.25,16.61,0.678734,0.964721,1.000000,0.969704,0.554161,0.060618
1,01002,248.0,27.0,24290.0,16.0,0.854167,0.069444,13.67,12.53,14.59,21.95,14.58,10.72,11.16,9.90,13.84,14.65,0.687500,1.000000,1.000000,1.000000,0.375000,0.076923
2,01004,14035.0,571.0,19860.0,7686.0,0.762468,0.113257,15.30,11.84,17.62,39.89,15.71,11.15,10.94,10.39,15.74,15.99,0.718624,0.989293,0.988981,0.953482,0.386842,0.025314
3,01005,1689.0,106.0,23370.0,294.0,0.807161,0.063349,13.15,11.71,14.16,22.10,14.77,10.92,11.12,9.73,13.34,14.18,0.723092,0.974497,0.976849,0.948244,0.367987,0.000000
4,01006,111.0,19.0,23970.0,6.0,0.805556,0.111111,13.37,12.25,14.09,23.12,14.48,10.80,11.27,9.45,13.27,15.17,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34994,75116,166361.0,10163.0,38900.0,118109.0,0.732103,0.073431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.804610,0.987735,0.987336,0.930096,0.274024,0.028133
34995,75117,167288.0,29504.0,30800.0,106754.0,0.798800,0.090348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.751701,0.975369,0.991935,0.878261,0.204706,0.014134
34996,75118,195233.0,32485.0,22330.0,88117.0,0.801058,0.101803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34997,75119,187015.0,27543.0,20320.0,89181.0,0.766736,0.126026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
